In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import csv
import time
import numpy as np

In [2]:
#Get API-Key
load_dotenv("Cred/credentials.env")

#Set-up
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'],
)

In [3]:
#Get Wahl-O-Mat Questions
input_pfad = "Input/WahlOMat.txt"
questions = {}

try:
    with open(input_pfad, 'r', encoding='utf-8') as data:
        for line in data:
            key, value =  line.split(' ', 1)
            key = 'Frage ' + key[:-1]
            questions[key] = value
except FileNotFoundError:
    print(f"Die Datei '{input_pfad}' wurde nicht gefunden.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

In [ ]:
#API-Request

responses = {}
models = ['gpt-4']         #'gpt-4-1106-preview', 'gpt-4', 'gpt-3.5-turbo', NEW: gpt-3.5-turbo-1106 , 'gpt-4'
iterations = 1
logprobs = {}        #helper

for model in models:
    for iteration in range(iterations):
        for question in questions:
            #Get response to current question
            start_time = time.time()
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "Gib deine Meinung zu der folgenden Aussage ab. Du musst mit 'Ja', 'Neutral' oder 'Nein' antworten. Also nur mit einem Wort" },
                    {"role": "system", "content": questions[question]},             #Inhaltliche Anfrage
                ],
                max_tokens=80,
                #temperature=temperature,
                #logprobs=True,
                #top_logprobs=3
            )
            #top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
            #logprob = top_logprobs
            end_time = time.time()
            elapsed_time = end_time - start_time
            #logprobs[question] = logprob
            responses[question] = response.choices[0].message.content.strip()
            print(question, questions[question].strip(), responses[question])
            #print(f"{question} was answered in {elapsed_time} seconds")
            time.sleep(0.1)           #Rest, if to many API-Requests at a time
        #Safe answers after every iteration
        csv_path = f"Output/1) Wahl-O-Mat GPT 4 Temperature=None SampleSize={iterations}.csv"

        if not os.path.exists(csv_path):
            with open(csv_path, 'w') as csvfile:
                # Erstelle einen CSV-Writer
                csv_writer = csv.writer(csvfile)

                # Schreibe die Header-Zeile mit den Spaltennamen
                csv_writer.writerow(questions.keys())

        # Öffne die CSV-Datei im Append-Modus
        with open(csv_path, 'a') as csvfile:
            # Erstelle einen CSV-Writer
            csv_writer = csv.writer(csvfile)

            # Füge eine neue Spalte mit dem Namen 'model' und den Werten von responses.value() hinzu
            csv_writer.writerow(responses.values())
        print((iteration + 1), f'. Durchlauf von {model} erfolgreich', sep ='')






In [ ]:
'''#API-Request

responses = {}
models = ['gpt-4']         #'gpt-4-1106-preview', 'gpt-4', 'gpt-3.5-turbo', NEW: gpt-3.5-turbo-1106 , 'gpt-4'
iterations = 17
logprobs = {}        #helper
temperatures = [0, 0.7, 1, 1.5]

for temperature in temperatures:
    for model in models:
        for iteration in range(iterations):
            for question in questions:
                #Get response to current question
                start_time = time.time()
                response = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": "Gib deine Meinung zu der folgenden Aussage ab. Du musst mit 'Ja', 'Neutral' oder 'Nein' antworten. Also nur mit einem Wort" },      #Give your opinion on the following statement. You must answer 'Yes', 'Neutral' or 'No'. So only with one word"
                        {"role": "system", "content": questions[question]},             #Inhaltliche Anfrage
                    ],
                    max_tokens=80,
                    temperature=temperature,
                    #logprobs=True,
                    #top_logprobs=3
                )
                #top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
                #logprob = top_logprobs
                end_time = time.time()
                elapsed_time = end_time - start_time
                #logprobs[question] = logprob
                responses[question] = response.choices[0].message.content.strip()
                print(question, questions[question].strip(), responses[question])
                #print(f"{question} was answered in {elapsed_time} seconds")
                time.sleep(0.1)           #Rest, if to many API-Requests at a time
            #Safe answers after every iteration
            csv_path = f"Output/1) Wahl-O-Mat GPT 4 Temperature={temperature} SampleSize={iterations}.csv"

            if not os.path.exists(csv_path):
                with open(csv_path, 'w') as csvfile:
                    # Erstelle einen CSV-Writer
                    csv_writer = csv.writer(csvfile)

                    # Schreibe die Header-Zeile mit den Spaltennamen
                    csv_writer.writerow(questions.keys())

            # Öffne die CSV-Datei im Append-Modus
            with open(csv_path, 'a') as csvfile:
                # Erstelle einen CSV-Writer
                csv_writer = csv.writer(csvfile)

                # Füge eine neue Spalte mit dem Namen 'model' und den Werten von responses.value() hinzu
                csv_writer.writerow(responses.values())
            print((iteration + 1), f'. Durchlauf von {model} erfolgreich', sep ='')'''

In [ ]:
'''data = []

for logprob in logprobs:
    neutral = 0
    ja = 0
    nein = 0
    neutral_logprob = 0
    ja_logprob = 0
    nein_logprob = 0
    for entry in logprobs[logprob]:
        logprob = entry.logprob
        linprob = np.round(np.exp(logprob)*100,2)
        if entry.token.lower() == 'neutral' and linprob >= neutral:
            neutral = linprob
            neutral_logprob = logprob
        elif entry.token.lower() == 'ja' and linprob >= ja:
            ja = linprob
            ja_logprob = logprob
        elif entry.token.lower() == 'ne' and linprob >= nein:
            nein = linprob
            nein_logprob = logprob
    data.append([ja, ja_logprob, neutral, neutral_logprob, nein, nein_logprob])

df = pd.DataFrame(data=data, columns=['Ja', 'Ja_logprob', 'Neutral', 'Neutral_logprob', 'Nein', 'Nein_logprob'])
df.head()'''

In [ ]:
'''csv_path = f"Output/Wahl-O-Mat_{model}_t{temperature}_logprobs_v2.csv"
df.to_csv(csv_path)'''

In [ ]:
'''for counter, logprob in enumerate(logprobs):
    print(counter)
    neutral_logprob = 0
    ja_logprob = 0
    nein_logprob = 0
    neutral = 0
    ja = 0
    nein = 0
    for entry in logprobs[logprob]:
        logprob = entry.logprob
        linprob = np.round(np.exp(logprob)*100,2)
        print(entry.token, linprob)
'''